## Library Imports, Data Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import itertools
from sklearn.metrics import silhouette_score
import networkx as nx
from collections import defaultdict
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn_extra.cluster import KMedoids
from itertools import combinations
from statsmodels.stats.anova import anova_lm





### Data import section is slightly altered to cover any specific references to the firm

In [ ]:
# Import orderlines
OrderLinesData = pd.read_csv('data/Symson Orderlines 20241121.csv', encoding='ISO-8859-1', sep=';', decimal=',')

# Import product data
Products = pd.read_csv('data/Symson Products 20241121.csv', encoding='ISO-8859-1', sep=';', decimal=',')

# Rename columns in Products
Products.rename(columns={'ProductAttributeC':'PriceChangeFrequency','ProductAttributeE':'CompetitorIntensity', 'productId':'ProductId',
                         'ProductAttributeF':'ProductType','ProductAttributeB':'Availability','ProductAttributeG':'Sustainability'}, inplace=True)

# Rename deals
OrderLinesData.rename(columns={'OrderLineAttributeD':'Deals'}, inplace=True)

# New OrderLines
NewOrderLines = pd.read_csv('data/Orderlines.csv', encoding='ISO-8859-1', sep=';', decimal=',')

# Rename orderline attributes ProjectDiscount and Matrix
NewOrderLines.rename(columns={'OrderlineAttributeE':'ProjectDiscount','OrderlineAttributeF':'Matrix'}, inplace=True)

## Data Processing & Attribute Creation

In [ ]:
OrderLinesData = OrderLinesData[(OrderLinesData['PricePerUnit']>0) & (OrderLinesData['Quantity']>=1)]

In [ ]:
OrderLinesData = OrderLinesData.drop_duplicates(subset=['InvoiceId', 'ProductId'])

In [ ]:
# Month
OrderLinesData['Date Month'] = pd.to_datetime(OrderLinesData['InvoiceCreationDate']).dt.to_period('M')

# Availability
Products['Availability'] = np.where(Products['Availability']=='ON STOCK                                ', 1, 0)

## Regression Preparation and Execution

In [ ]:
Quantity = OrderLinesData.groupby(["ProductId", "PricePerUnit", "InvoiceCreationDate"])['Quantity'].sum().reset_index()

In [ ]:
# Quantity
Quantity = OrderLinesData.filter(["ProductId", "Quantity",'Date Month'])
Quantity = Quantity.groupby(["ProductId",'Date Month']).sum().reset_index()

# Price
Price = OrderLinesData.filter(['ProductId','Date Month'])
Price.drop_duplicates(inplace=True)
W_Average_Prices = OrderLinesData.groupby(['ProductId','Date Month']).apply(lambda x: np.sum(x['Quantity']*x['PricePerUnit'])/x['Quantity'].sum(), include_groups=False).reset_index(name='PricePerUnit')

In [ ]:
num_customers = OrderLinesData.groupby("ProductId")["CustomerId"].nunique().rename("NumCustomers").reset_index()

In [ ]:
dfRegression = pd.merge(Quantity, W_Average_Prices, on=['ProductId','Date Month'], how='left') 
dfRegression = pd.merge(dfRegression, OrderLinesData[['ProductId','Date Month']].drop_duplicates(), on=['ProductId','Date Month'], how='left')
dfRegression = pd.merge(dfRegression, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
dfRegression = pd.merge(dfRegression, num_customers, on='ProductId', how='left')
dfRegression.dropna(inplace=True)

In [ ]:
len(OrderLinesData)

In [ ]:
OrderLinesData["ProductId"].nunique()

In [ ]:
dfRegression_t = dfRegression.copy()

In [ ]:
dfRegression_t["MainProduct"] = np.where(dfRegression_t["ProductType"] == "HOOFDPRODUCT", 1, 0)

# KVI Work

### Slightly concealed to hide firm-specific references

In [ ]:
# Quantity
Quantity = OrderLinesData.filter(["ProductId", "Quantity"])
Quantity = Quantity.groupby(["ProductId"]).sum().reset_index()

# Price
Price = OrderLinesData.filter(['ProductId'])
Price.drop_duplicates(inplace=True)
W_Average_Prices = OrderLinesData.groupby(['ProductId']).apply(lambda x: np.sum(x['Quantity']*x['PricePerUnit'])/x['Quantity'].sum(), include_groups=False).reset_index(name='PricePerUnit')

In [ ]:
dfSensitivity = pd.merge(Quantity, W_Average_Prices, on=['ProductId'], how='left') 
dfSensitivity = pd.merge(dfSensitivity, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
dfSensitivity = pd.merge(dfSensitivity, num_customers, on='ProductId', how='left')

In [ ]:
dfSensitivity

In [ ]:
KVI_Analysis = pd.read_csv('data/KVI_List.csv')
KVI_Analysis = KVI_Analysis[KVI_Analysis["Category"] == "KVI"]
KVI_list = KVI_Analysis["ProductId"].tolist()
dfSensitivity["KVI_dummy"] = dfSensitivity["ProductId"].isin(KVI_list).astype(int)


## KVI Clustering

In [ ]:
dfAllStandard = dfSensitivity.copy()

In [ ]:
dfAllStandard = dfAllStandard.dropna()

In [ ]:
product_purchase_count = OrderLinesData["ProductId"].value_counts().reset_index()
product_purchase_count.columns = ["ProductId", "PurchaseCount"]
dfAllStandard= dfAllStandard.merge(product_purchase_count, on="ProductId", how="left")

In [ ]:
dfAllStandard["ProductType"] = np.where(dfAllStandard["ProductType"] == "HOOFDPRODUCT", 1, 0)

In [ ]:
dfAllStandard["Revenue"] = dfAllStandard["Quantity"] * dfAllStandard["PricePerUnit"]

In [ ]:


df_standard_product_ids = dfAllStandard['ProductId'].tolist()
basket_groups = OrderLinesData.groupby('InvoiceId')['ProductId'].apply(list)

co_purchase_counts = defaultdict(int)
product_counts = defaultdict(int)  

for basket in basket_groups:
    filtered_basket = [prod for prod in basket if prod in df_standard_product_ids]
    for i in range(len(filtered_basket)):
        product_counts[filtered_basket[i]] += 1  
        for j in range(i + 1, len(filtered_basket)):
            pair = tuple(sorted([filtered_basket[i], filtered_basket[j]]))  
            co_purchase_counts[pair] += 1


G = nx.Graph()

for (product1, product2), weight in co_purchase_counts.items():
    G.add_edge(product1, product2, weight=weight)

pos = nx.spring_layout(G, seed=42, k=5) 

edge_colors = []
edge_widths = []

max_weight = max(co_purchase_counts.values())

for u, v, d in G.edges(data=True):
    weight = d["weight"]
    edge_widths.append(weight / max_weight * 12)  
    

plt.figure(figsize=(35, 35))

nx.draw_networkx_nodes(G, pos, edgecolors="black")

nx.draw_networkx_edges(G, pos, alpha=1, width=edge_widths)

G.remove_edges_from(nx.selfloop_edges(G))

plt.title("Product Co-Purchase Network")
plt.show()


In [ ]:
degree_centrality = nx.degree_centrality(G)

In [ ]:
dfAllStandard["DegreeCentrality"] = dfAllStandard["ProductId"].map(degree_centrality)

In [ ]:
#Takes 45 minutes to run, stored in csv for easier access
#betweenness_centrality = nx.betweenness_centrality(G, k=None, weight="weight")
# betweenness_centrality_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['ProductId', 'BetweennessCentrality'])
# betweenness_centrality_df.to_csv('betweenness_centrality.csv', index=False)
betweenness_centrality_df = pd.read_csv('data/betweenness_centrality.csv')


In [ ]:
dfAllStandard = dfAllStandard.merge(betweenness_centrality_df, on="ProductId", how="left")

In [ ]:
len(dfAllStandard)

In [ ]:
dfAllStandard.isna().sum()

In [ ]:
dfAllStandard = dfAllStandard.dropna()

In [ ]:
print(len(dfAllStandard))
dfAllStandard = dfAllStandard[dfAllStandard["BetweennessCentrality"] > 0]
print(len(dfAllStandard))

In [ ]:
df_sorted = dfAllStandard.sort_values(by='Revenue', ascending=False).reset_index(drop=True)

# Calculate cumulative revenue
df_sorted['CumulativeRevenue'] = df_sorted['Revenue'].cumsum()
df_sorted['CumulativeRevenue'] /= df_sorted['Revenue'].sum()  # Normalize to make it proportion

# Calculate x-axis as percentage of products
df_sorted['ProductPercent'] = np.linspace(0, 100, len(df_sorted))

# Plot
plt.figure(figsize=(8, 5))
plt.plot(df_sorted['ProductPercent'], df_sorted['CumulativeRevenue']*100, marker=',', linestyle='-')
plt.xlabel("Top Percentage of Products Sorted by Revenue")
plt.ylabel("Cumulative Revenue Share (%)")
plt.grid(True)
plt.show()

In [ ]:
cut_points = [0.5, 0.8, 0.9, 0.95, 0.96, 0.97, .98]

for cp in cut_points:
    idx = df_sorted[df_sorted['CumulativeRevenue'] >= cp].index[0]
    percent_of_products = df_sorted.loc[idx, 'ProductPercent']
    print(f"{cp*100:.0f}% of revenue is generated by the top {percent_of_products:.2f}% of products.")

In [ ]:
df_sorted[df_sorted["ProductPercent"] <= 25]

In [ ]:
len(df_sorted)

In [ ]:
dfAllStandard = dfAllStandard.sort_values(by='Revenue', ascending=False).head(round(len(dfAllStandard) / 4)-1)
print(len(dfAllStandard))

In [ ]:
dfAllStandard.columns

In [ ]:
dfAllStandard.dtypes

In [ ]:
dfAllStandard[["Quantity", "PricePerUnit", "Revenue", "NumCustomers", 
               "PurchaseCount", "Availability", "PriceChangeFrequency", 
               "ProductType", "DegreeCentrality", "BetweennessCentrality"]].describe()


In [ ]:
dfAllStandard["NumCustomers"] = np.log(dfAllStandard["NumCustomers"])
dfAllStandard["PurchaseCount"] = np.log(dfAllStandard["PurchaseCount"])
dfAllStandard["PricePerUnit"] = np.log(dfAllStandard["PricePerUnit"])
dfAllStandard["DegreeCentrality"] = np.log(dfAllStandard["DegreeCentrality"])
dfAllStandard["BetweennessCentrality"] = np.log(dfAllStandard["BetweennessCentrality"])
dfAllStandard["Revenue"] = np.log(dfAllStandard["Revenue"])
dfAllStandard["Quantity"] = np.log(dfAllStandard["Quantity"])


In [ ]:

features_to_scale = ['PricePerUnit', 'PurchaseCount', 'NumCustomers', 
                     'DegreeCentrality', 'BetweennessCentrality','Revenue', 'Quantity']

scaler = RobustScaler()

for feature in features_to_scale:
    dfAllStandard[feature + '_Scaled'] = scaler.fit_transform(dfAllStandard[[feature]])

In [ ]:
dfAllStandard

In [ ]:

features = [
    'Quantity_Scaled', 'Revenue_Scaled',
    'NumCustomers_Scaled', 'PurchaseCount_Scaled', 'PricePerUnit_Scaled'
]

X = dfAllStandard[features].values  

# 1. Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# 2. Explained variance plot (Scree plot)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 
         marker='o', linestyle='--')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid()
plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1)) 
plt.show()



In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Set number of components for PCA
n_components = 3

# Reduce dimensionality using PCA
pca = PCA(n_components=n_components)
X_pca_reduced = pca.fit_transform(X)

# 1. Apply K-Means Clustering
wcss_kmeans = []  # Within-cluster sum of squares for K-Means
silhouette_scores_kmeans = []  # Silhouette scores for K-Means
K_range = range(2, 60)  # K-means clustering from 2 to 39 clusters

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca_reduced)
    wcss_kmeans.append(kmeans.inertia_)  # WCSS for K-means
    
    # Compute silhouette score for K-means
    silhouette_avg_kmeans = silhouette_score(X_pca_reduced, kmeans.labels_)
    silhouette_scores_kmeans.append(silhouette_avg_kmeans)

# 2. Apply K-Medoids Clustering
wcss_kmedoids = []  # Within-cluster sum of squares for K-Medoids
silhouette_scores_kmedoids = []  # Silhouette scores for K-Medoids

for k in K_range:
    kmedoids = KMedoids(n_clusters=k, random_state=42)
    kmedoids.fit(X_pca_reduced)
    wcss_kmedoids.append(kmedoids.inertia_)  # WCSS for K-medoids
    
    # Compute silhouette score for K-medoids
    silhouette_avg_kmedoids = silhouette_score(X_pca_reduced, kmedoids.labels_)
    silhouette_scores_kmedoids.append(silhouette_avg_kmedoids)

# 3. Plot WCSS for K-means and K-medoids
plt.figure(figsize=(10, 6))
plt.plot(K_range, wcss_kmeans, marker='o', linestyle='--', label='K-Means WCSS', color='blue')
plt.plot(K_range, wcss_kmedoids, marker='s', linestyle='--', label='K-Medoids WCSS', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("WCSS (Within-cluster Sum of Squares)")
plt.legend()
plt.grid(True)
plt.show()

# 4. Plot Silhouette Scores for K-means and K-medoids
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores_kmeans, marker='o', linestyle='-', label='K-Means Silhouette Score', color='blue')
plt.plot(K_range, silhouette_scores_kmedoids, marker='s', linestyle='-', label='K-Medoids Silhouette Score', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Set number of components for PCA
n_components = 3

# Reduce dimensionality using PCA
pca = PCA(n_components=n_components)
X_pca_reduced = pca.fit_transform(X)

optimal_k = 20  # Adjust based on the plot

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca_reduced)

# Assign clusters to the original dataset
dfAllStandard["Cluster"] = clusters

cluster_distribution = dfAllStandard['Cluster'].value_counts(normalize=True).mul(100).round(2)
print("Cluster Size (% of Total):\n", cluster_distribution)

print(dfAllStandard["Cluster"].value_counts())

# 5. Analyze clusters by computing feature means per cluster
cluster_summary = dfAllStandard.groupby("Cluster")[features].mean()
cluster_summary.head(30)



In [ ]:
cluster= 15
print(f"{dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()}/{dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]} = {dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()/dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]}")

In [ ]:

features = [
    'Quantity_Scaled', 'Revenue_Scaled',
    'NumCustomers_Scaled', 'PurchaseCount_Scaled', 'DegreeCentrality_Scaled', 'PricePerUnit_Scaled', 'BetweennessCentrality_Scaled'
]

X = dfAllStandard[features].values  

# 1. Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# 2. Explained variance plot (Scree plot)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 
         marker='o', linestyle='--')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid()
plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1)) 
plt.show()



In [ ]:
pca.explained_variance_ratio_

In [ ]:
loadings = pd.DataFrame(pca.components_.T, 
                        columns=[f'PC{i+1}' for i in range(pca.n_components_)],
                        index=features)

plt.figure(figsize=(10, 6))

# Create the heatmap
sns.heatmap(loadings[["PC1", "PC2", "PC3"]], annot=True, cmap='coolwarm', center=0)

# Add labels and title
plt.xlabel('Principal Component')
plt.ylabel('Features')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
loadings

In [ ]:
# Set number of components for PCA
n_components = 3

# Reduce dimensionality using PCA
pca = PCA(n_components=n_components)
X_pca_reduced = pca.fit_transform(X)

# 1. Apply K-Means Clustering
wcss_kmeans = []  # Within-cluster sum of squares for K-Means
silhouette_scores_kmeans = []  # Silhouette scores for K-Means
K_range = range(2, 60)  # K-means clustering from 2 to 39 clusters

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca_reduced)
    wcss_kmeans.append(kmeans.inertia_)  # WCSS for K-means
    
    # Compute silhouette score for K-means
    silhouette_avg_kmeans = silhouette_score(X_pca_reduced, kmeans.labels_)
    silhouette_scores_kmeans.append(silhouette_avg_kmeans)

# 2. Apply K-Medoids Clustering
wcss_kmedoids = []  # Within-cluster sum of squares for K-Medoids
silhouette_scores_kmedoids = []  # Silhouette scores for K-Medoids

for k in K_range:
    kmedoids = KMedoids(n_clusters=k, random_state=42)
    kmedoids.fit(X_pca_reduced)
    wcss_kmedoids.append(kmedoids.inertia_)  # WCSS for K-medoids
    
    # Compute silhouette score for K-medoids
    silhouette_avg_kmedoids = silhouette_score(X_pca_reduced, kmedoids.labels_)
    silhouette_scores_kmedoids.append(silhouette_avg_kmedoids)

# 3. Plot WCSS for K-means and K-medoids
plt.figure(figsize=(10, 6))
plt.plot(K_range, wcss_kmeans, marker='o', linestyle='--', label='K-Means WCSS', color='blue')
plt.plot(K_range, wcss_kmedoids, marker='s', linestyle='--', label='K-Medoids WCSS', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("WCSS (Within-cluster Sum of Squares)")
plt.legend()
plt.grid(True)
plt.show()

# 4. Plot Silhouette Scores for K-means and K-medoids
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores_kmeans, marker='o', linestyle='-', label='K-Means Silhouette Score', color='blue')
plt.plot(K_range, silhouette_scores_kmedoids, marker='s', linestyle='-', label='K-Medoids Silhouette Score', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 3. Apply K-Means Clustering (Optimal k selection using Elbow Method)
wcss = []  # Within-cluster sum of squares
silhouette_scores = []  # Silhouette scores
K_range = range(2, 40) 

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca_reduced)
    wcss.append(kmeans.inertia_)
    
    # Compute silhouette score
    silhouette_avg = silhouette_score(X_pca_reduced, kmeans.labels_)
    silhouette_scores.append(silhouette_avg)

# Plot the Elbow Method (WCSS)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(K_range, wcss, marker='o', linestyle='--', label='WCSS')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("WCSS (Within-cluster Sum of Squares)")
plt.title("Elbow Method for Optimal k")
plt.grid()
plt.legend()

# Plot the Silhouette Score
plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, marker='s', linestyle='-', color='red', label='Silhouette Score')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score for Different k Values")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
optimal_k = 20  # Adjust based on the plot

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_pca_reduced)

# Assign clusters to the original dataset
dfAllStandard["Cluster"] = clusters

cluster_distribution = dfAllStandard['Cluster'].value_counts(normalize=True).mul(100).round(2)
print("Cluster Size (% of Total):\n", cluster_distribution)

print(dfAllStandard["Cluster"].value_counts())

# 5. Analyze clusters by computing feature means per cluster
cluster_summary = dfAllStandard.groupby("Cluster")[features].mean()
cluster_summary.head(30)



In [ ]:
cluster= 10
print(f"{dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()}/{dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]} = {dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()/dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]}")

In [ ]:
for cluster in range (0, 20):
    print(len(dfAllStandard[dfAllStandard['Cluster'] == cluster]))


In [ ]:
for cluster in range (0, 20):
    print(f"{dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()}/{dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]} = {dfAllStandard[dfAllStandard['Cluster'] == cluster]['KVI_dummy'].sum()/dfAllStandard[dfAllStandard['Cluster'] == cluster].shape[0]}")

In [ ]:
dfAllStandard["KVI_dummy"].sum()

In [ ]:
dfAllStandard[dfAllStandard["KVI_dummy"] == 1][features].describe()

In [ ]:
dfAllStandard[dfAllStandard["KVI_dummy"] == 0][features].describe()

In [ ]:
dfAllStandard[dfAllStandard["KVI_dummy"] == 1][features].sort_values("Revenue_Scaled", ascending=False).head(56).describe()

In [ ]:
dfAllStandard[dfAllStandard["KVI_dummy"] == 1].head(56)["Cluster"].value_counts()

In [ ]:
dfAllStandard.head(56)["Cluster"].value_counts()

In [ ]:
dfAllStandard[dfAllStandard["KVI_dummy"] == 1].iloc[56:213]["Cluster"].value_counts()

In [ ]:

# Create masks
kvi_mask = dfAllStandard["KVI_dummy"] == 1
non_kvi_mask = ~kvi_mask
cluster_6_mask = dfAllStandard["Cluster"] == 10
non_cluster_6_mask = ~cluster_6_mask

# Use only first 2 PCA components
reduced_data_2d = X_pca_reduced[:, :2]

fig, ax = plt.subplots(figsize=(10, 7))

# Plot non-cluster 6 points (all grey)
# Circles for non-KVI
ax.scatter(reduced_data_2d[non_cluster_6_mask & non_kvi_mask, 0],
           reduced_data_2d[non_cluster_6_mask & non_kvi_mask, 1],
           color='lightgrey', alpha=0.6, label='Other - Non-KVI', marker='o')


# Plot cluster 6 points (orange)
# Circles for non-KVI
ax.scatter(reduced_data_2d[cluster_6_mask & non_kvi_mask, 0],
           reduced_data_2d[cluster_6_mask & non_kvi_mask, 1],
           color='orange', alpha=0.9, label='Cluster 10 - Non-KVI', marker='o')

# Triangles for KVI - outside cluster 6 (grey)
ax.scatter(reduced_data_2d[non_cluster_6_mask & kvi_mask, 0],
           reduced_data_2d[non_cluster_6_mask & kvi_mask, 1],
           color='lightgrey', edgecolors='black', linewidths=.5,
           alpha=0.6, label='Other - KVI', marker='^')

# Triangles for KVI - inside cluster 6 (orange)
ax.scatter(reduced_data_2d[cluster_6_mask & kvi_mask, 0],
           reduced_data_2d[cluster_6_mask & kvi_mask, 1],
           color='orange', edgecolors='black', linewidths=.5,
           alpha=0.9, label='Cluster 10 - KVI', marker='^')

ax.set_xlabel("PCA Component 1")
ax.set_ylabel("PCA Component 2")

# Avoid duplicate labels in legend
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

plt.tight_layout()
plt.show()


In [ ]:
# Calculate the total number of KVI_dummy = 1 rows
total_kvi = dfAllStandard[dfAllStandard['KVI_dummy'] == 1].shape[0]

# Calculate the number of KVI_dummy = 1 rows for each cluster
kvi_per_cluster = dfAllStandard[dfAllStandard['KVI_dummy'] == 1]['Cluster'].value_counts()

# Calculate the proportion of KVI_dummy = 1 rows for each cluster
kvi_proportion_per_cluster = (kvi_per_cluster / total_kvi) * 100

# Display the results
print(kvi_proportion_per_cluster)

In [ ]:
# Calculate the total number of rows per cluster
total_rows_per_cluster = dfAllStandard['Cluster'].value_counts()

# Calculate the number of KVI_dummy = 1 rows per cluster
kvi_rows_per_cluster = dfAllStandard[dfAllStandard['KVI_dummy'] == 1]['Cluster'].value_counts()

# Calculate the accuracy per cluster
accuracy_per_cluster = (kvi_rows_per_cluster / total_rows_per_cluster) * 100

# Display the results
print(accuracy_per_cluster)

In [ ]:
cluster_number = 10
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == cluster_number]["ProductId"].to_list()


invoice_products = OrderLinesData.groupby('InvoiceId')['ProductId'].apply(list)


pair_list = []
for product_list in invoice_products:
    pairs = combinations(sorted(set(product_list)), 2)  # remove duplicates in invoice
    pair_list.extend(pairs)

pair_df = pd.DataFrame(pair_list, columns=['Product_A', 'Product_B'])

pair_counts = pair_df.value_counts().reset_index(name='Count')


In [ ]:
pair_counts["KVI_A"] = pair_counts["Product_A"].isin(KVI_Cluster).astype(int)
pair_counts["KVI_B"] = pair_counts["Product_B"].isin(KVI_Cluster).astype(int)

In [ ]:
pair_counts = pair_counts[pair_counts["KVI_A"] + pair_counts["KVI_B"] > 0]

In [ ]:
pair_counts

In [ ]:
OrderLinesData["InvoiceCreationDate"].nunique()

In [ ]:
pair_counts = pair_counts[pair_counts["Count"] >= 161]

In [ ]:
pd.concat([
    pd.Series(pair_counts[pair_counts["KVI_B"] == 1]["Product_B"].unique()),
    pd.Series(pair_counts[pair_counts["KVI_A"] == 1]["Product_A"].unique())
]).nunique()


In [ ]:


mask = (pair_counts["KVI_A"] == 0) & (pair_counts["KVI_B"] == 1)

pair_counts.loc[mask, ["Product_A", "Product_B"]] = pair_counts.loc[mask, ["Product_B", "Product_A"]].values
pair_counts.loc[mask, ["KVI_A", "KVI_B"]] = pair_counts.loc[mask, ["KVI_B", "KVI_A"]].values


In [ ]:
pair_counts

In [ ]:


results = []

for _, row in pair_counts.iterrows():
    prod_a = row['Product_A']
    prod_b = row['Product_B']
    
    # Get all invoices containing A
    invoices_a = OrderLinesData[OrderLinesData['ProductId'] == prod_a][['InvoiceId', 'PricePerUnit']]
    invoices_a = invoices_a.rename(columns={'PricePerUnit': 'Price_A'})

    # Get all invoices containing B
    invoices_b = OrderLinesData[OrderLinesData['ProductId'] == prod_b][['InvoiceId', 'Quantity']]
    invoices_b = invoices_b.rename(columns={'Quantity': 'Quantity_B'})
    
    # Merge on InvoiceId to get only those invoices where both appear
    merged = invoices_a.merge(invoices_b, on='InvoiceId')

    if len(merged) >= 10:
        merged = merged[(merged['Quantity_B'] > 0) & (merged['Price_A'] > 0)]

        # Log-transform
        merged['log_Q_B'] = np.log(merged['Quantity_B'])
        merged['log_P_A'] = np.log(merged['Price_A'])

        # Add constant for intercept
        X = sm.add_constant(merged['log_P_A'])
        y = merged['log_Q_B']

        # Fit regression
        model = sm.OLS(y, X).fit()

        elasticity = model.params['log_P_A']
        p_value = model.pvalues['log_P_A']
        unique_prices = merged['Price_A'].nunique()
        unique_quantities = merged["Quantity_B"].nunique()
        
        results.append({
            'Product_A': prod_a,
            'Product_B': prod_b,
            'Cross_Elasticity': elasticity,
            'P_Value': p_value,
            'Num_Samples': len(merged),
            'Unique_Prices_A': unique_prices,
            'Unique_Quantities_B': unique_quantities
        })


In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
pair_counts = pair_counts.merge(results_df.filter(["Product_A", "Product_B", "Cross_Elasticity", "P_Value", "Unique_Prices_A", "Unique_Quantities_B"]), on=["Product_A", "Product_B"], how="left")

In [ ]:
def cross_price_elasticity(cluster_num, threshold=161):
    KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == cluster_num]["ProductId"].to_list()
    invoice_products = OrderLinesData.groupby('InvoiceId')['ProductId'].apply(list)
    pair_list = []
    for product_list in invoice_products:
        pairs = combinations(sorted(set(product_list)), 2)  # remove duplicates in invoice
        pair_list.extend(pairs)

    pair_df = pd.DataFrame(pair_list, columns=['Product_A', 'Product_B'])

    pair_counts = pair_df.value_counts().reset_index(name='Count')
    pair_counts["KVI_A"] = pair_counts["Product_A"].isin(KVI_Cluster).astype(int)
    pair_counts["KVI_B"] = pair_counts["Product_B"].isin(KVI_Cluster).astype(int)
    pair_counts = pair_counts[pair_counts["KVI_A"] + pair_counts["KVI_B"] > 0]
    pair_counts = pair_counts[pair_counts["Count"] >= threshold]

    if len(pair_counts) == 0:
        return "No pairs"
    pd.concat([
        pd.Series(pair_counts[pair_counts["KVI_B"] == 1]["Product_B"].unique()),
        pd.Series(pair_counts[pair_counts["KVI_A"] == 1]["Product_A"].unique())
    ]).nunique()
    mask = (pair_counts["KVI_A"] == 0) & (pair_counts["KVI_B"] == 1)

    pair_counts.loc[mask, ["Product_A", "Product_B"]] = pair_counts.loc[mask, ["Product_B", "Product_A"]].values
    pair_counts.loc[mask, ["KVI_A", "KVI_B"]] = pair_counts.loc[mask, ["KVI_B", "KVI_A"]].values


    results = []

    for _, row in pair_counts.iterrows():
        prod_a = row['Product_A']
        prod_b = row['Product_B']
        
        # Get all invoices containing A
        invoices_a = OrderLinesData[OrderLinesData['ProductId'] == prod_a][['InvoiceId', 'PricePerUnit']]
        invoices_a = invoices_a.rename(columns={'PricePerUnit': 'Price_A'})

        # Get all invoices containing B
        invoices_b = OrderLinesData[OrderLinesData['ProductId'] == prod_b][['InvoiceId', 'Quantity']]
        invoices_b = invoices_b.rename(columns={'Quantity': 'Quantity_B'})
        
        # Merge on InvoiceId to get only those invoices where both appear
        merged = invoices_a.merge(invoices_b, on='InvoiceId')

        if len(merged) >= 10:
            merged = merged[(merged['Quantity_B'] > 0) & (merged['Price_A'] > 0)]

            # Log-transform
            merged['log_Q_B'] = np.log(merged['Quantity_B'])
            merged['log_P_A'] = np.log(merged['Price_A'])

            # Add constant for intercept
            X = sm.add_constant(merged['log_P_A'])
            y = merged['log_Q_B']

            # Fit regression
            # Fit regression with clustered standard errors by InvoiceId
            model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': merged['InvoiceId']})


            elasticity = model.params['log_P_A']
            p_value = model.pvalues['log_P_A']
            unique_prices = merged['Price_A'].nunique()
            unique_quantities = merged["Quantity_B"].nunique()
            
            results.append({
                'Product_A': prod_a,
                'Product_B': prod_b,
                'Cross_Elasticity': elasticity,
                'P_Value': p_value,
                'Num_Samples': len(merged),
                'Unique_Prices_A': unique_prices,
                'Unique_Quantities_B': unique_quantities
            })

    results_df = pd.DataFrame(results)
    pair_counts = pair_counts.merge(results_df.filter(["Product_A", "Product_B", "Cross_Elasticity", "P_Value", "Unique_Prices_A", "Unique_Quantities_B"]), on=["Product_A", "Product_B"], how="left")
    # Filter the DataFrame first
    filtered = pair_counts[(pair_counts["P_Value"] < 0.05) & (pair_counts["Unique_Prices_A"] > 10)]
    if len(filtered) == 0:
        return "No pairs"

    # Compute weighted average of Cross_Elasticity using 'Count' as weights
    weighted_avg = np.average(filtered["Cross_Elasticity"], weights=filtered["Count"])

    print(f"Cluster {cluster_num} Weighted Average Cross-Elasticity:", weighted_avg, "Average Cross-Elasticity:", filtered["Cross_Elasticity"].mean(), "Number of Pairs:", len(filtered))

for n in range(0,20):
    cross_price_elasticity(n, 161)
    

In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["KVI_dummy"] == 1]["ProductId"].to_list()
invoice_products = OrderLinesData.groupby('InvoiceId')['ProductId'].apply(list)
pair_list = []
for product_list in invoice_products:
    pairs = combinations(sorted(set(product_list)), 2)  # remove duplicates in invoice
    pair_list.extend(pairs)

pair_df = pd.DataFrame(pair_list, columns=['Product_A', 'Product_B'])

pair_counts = pair_df.value_counts().reset_index(name='Count')
pair_counts["KVI_A"] = pair_counts["Product_A"].isin(KVI_Cluster).astype(int)
pair_counts["KVI_B"] = pair_counts["Product_B"].isin(KVI_Cluster).astype(int)
pair_counts = pair_counts[pair_counts["KVI_A"] + pair_counts["KVI_B"] > 0]
pair_counts = pair_counts[pair_counts["Count"] >= 161]

if len(pair_counts) == 0:
    print("No pairs")
pd.concat([
    pd.Series(pair_counts[pair_counts["KVI_B"] == 1]["Product_B"].unique()),
    pd.Series(pair_counts[pair_counts["KVI_A"] == 1]["Product_A"].unique())
]).nunique()
mask = (pair_counts["KVI_A"] == 0) & (pair_counts["KVI_B"] == 1)

pair_counts.loc[mask, ["Product_A", "Product_B"]] = pair_counts.loc[mask, ["Product_B", "Product_A"]].values
pair_counts.loc[mask, ["KVI_A", "KVI_B"]] = pair_counts.loc[mask, ["KVI_B", "KVI_A"]].values


results = []

for _, row in pair_counts.iterrows():
    prod_a = row['Product_A']
    prod_b = row['Product_B']
    
    # Get all invoices containing A
    invoices_a = OrderLinesData[OrderLinesData['ProductId'] == prod_a][['InvoiceId', 'PricePerUnit']]
    invoices_a = invoices_a.rename(columns={'PricePerUnit': 'Price_A'})

    # Get all invoices containing B
    invoices_b = OrderLinesData[OrderLinesData['ProductId'] == prod_b][['InvoiceId', 'Quantity']]
    invoices_b = invoices_b.rename(columns={'Quantity': 'Quantity_B'})
    
    # Merge on InvoiceId to get only those invoices where both appear
    merged = invoices_a.merge(invoices_b, on='InvoiceId')

    if len(merged) >= 10:
        merged = merged[(merged['Quantity_B'] > 0) & (merged['Price_A'] > 0)]

        # Log-transform
        merged['log_Q_B'] = np.log(merged['Quantity_B'])
        merged['log_P_A'] = np.log(merged['Price_A'])

        # Add constant for intercept
        X = sm.add_constant(merged['log_P_A'])
        y = merged['log_Q_B']

        # Fit regression
        model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': merged['InvoiceId']})


        elasticity = model.params['log_P_A']
        p_value = model.pvalues['log_P_A']
        unique_prices = merged['Price_A'].nunique()
        unique_quantities = merged["Quantity_B"].nunique()
        
        results.append({
            'Product_A': prod_a,
            'Product_B': prod_b,
            'Cross_Elasticity': elasticity,
            'P_Value': p_value,
            'Num_Samples': len(merged),
            'Unique_Prices_A': unique_prices,
            'Unique_Quantities_B': unique_quantities
        })

results_df = pd.DataFrame(results)
pair_counts = pair_counts.merge(results_df.filter(["Product_A", "Product_B", "Cross_Elasticity", "P_Value", "Unique_Prices_A", "Unique_Quantities_B"]), on=["Product_A", "Product_B"], how="left")
# Filter the DataFrame first
filtered = pair_counts[(pair_counts["P_Value"] < 0.05) & (pair_counts["Unique_Prices_A"] > 10)]

# Compute weighted average of Cross_Elasticity using 'Count' as weights
weighted_avg = np.average(filtered["Cross_Elasticity"], weights=filtered["Count"])

print(f"Weighted Average Cross-Elasticity:", weighted_avg, "Average Cross-Elasticity:", filtered["Cross_Elasticity"].mean(), "Number of Pairs:", len(filtered))




In [ ]:
dfRegression_t['Quantity'] = np.log(dfRegression_t['Quantity'])
dfRegression_t['PricePerUnit'] = np.log(dfRegression_t['PricePerUnit'])
dfRegression_t['PriceChangeFrequency'] = np.log(dfRegression_t['PriceChangeFrequency']+1)
dfRegression_t['NumCustomers'] = np.log(dfRegression_t['NumCustomers'])


formula = 'Quantity ~ PricePerUnit*PriceChangeFrequency + PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers'

model_1 = smf.ols(formula=formula, data=dfRegression_t)
res_model_1 = model_1.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})

# Print the summary to check the results
print(res_model_1.summary())



In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 10]["ProductId"].to_list()
dfRegression_t["KVI"] = dfRegression_t["ProductId"].isin(KVI_Cluster).astype(int)
model_2 = smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVI' ,
                data=dfRegression_t)

res_2 = model_2.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})
print(res_2.summary())



In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 17]["ProductId"].to_list()
dfRegression_t["KVI"] = dfRegression_t["ProductId"].isin(KVI_Cluster).astype(int)
model_2_placebo_17 = smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVI' ,
                data=dfRegression_t)

res_2_placebo_17 = model_2_placebo_17.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})
print(res_2_placebo_17.summary())


In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 8]["ProductId"].to_list()
dfRegression_t["KVI"] = dfRegression_t["ProductId"].isin(KVI_Cluster).astype(int)
model_2_placebo_8 = smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVI' ,
                data=dfRegression_t)

res_2_placebo_8 = model_2_placebo_8.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})
print(res_2_placebo_8.summary())


In [ ]:
KVI_Cluster = dfAllStandard[(dfAllStandard["Cluster"] == 10) | (dfAllStandard["Cluster"] == 17) ]["ProductId"].to_list()
dfRegression_t["KVI"] = dfRegression_t["ProductId"].isin(KVI_Cluster).astype(int)
model_2_placebo_10_17 = smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVI' ,
                data=dfRegression_t)

res_2_placebo_10_17 = model_2_placebo_10_17.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})
print(res_2_placebo_10_17.summary())


In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["KVI_dummy"] == 1]["ProductId"].to_list()
dfRegression_t["KVI"] = dfRegression_t["ProductId"].isin(KVI_Cluster).astype(int)
model_2_placebo_list = smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVI' ,
                data=dfRegression_t)

res_2_placebo_list = model_2_placebo_list.fit(cov_type='cluster', cov_kwds={'groups': dfRegression_t['ProductId']})
print(res_2_placebo_list.summary())


In [ ]:

# Compare model_1 (baseline) to model_2
anova_results_2 = anova_lm(res_model_1, res_2)
print("ANOVA: model_1 vs model_2")
print(anova_results_2)

# Compare model_1 (baseline) to model_2_placebo_17
anova_results_placebo_17 = anova_lm(res_model_1, res_2_placebo_17)
print("\nANOVA: model_1 vs model_2_placebo_17")
print(anova_results_placebo_17)

# Compare model_1 (baseline) to model_2_placebo_19
anova_results_placebo_8 = anova_lm(res_model_1, res_2_placebo_8)
print("\nANOVA: model_1 vs model_2_placebo_8")
print(anova_results_placebo_8)


anova_results_placebo_10_17 = anova_lm(res_model_1, res_2_placebo_10_17)
print("\nANOVA: model_1 vs model_2_placebo_10_17")
print(anova_results_placebo_10_17)


anova_results_placebo_list = anova_lm(res_model_1, res_2_placebo_list)
print("\nANOVA: model_1 vs model_2_placebo_list")
print(anova_results_placebo_list)

In [ ]:
OrderLinesData.columns

In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 10]["ProductId"].to_list()
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_Cluster)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].min()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_3= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers' ,
                data=filtered_orderlines)

res_model_3 = model_3.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_3.summary())


In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 10]["ProductId"].to_list()
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_Cluster)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].max()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_4= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVIPrice' ,
                data=filtered_orderlines)

res_model_4 = model_4.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_4.summary())


In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 17]["ProductId"].to_list()
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_Cluster)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].max()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_17= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVIPrice' ,
                data=filtered_orderlines)

res_model_17 = model_17.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_17.summary())


In [ ]:
KVI_Cluster = dfAllStandard[dfAllStandard["Cluster"] == 8]["ProductId"].to_list()
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_Cluster)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].max()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_8= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVIPrice' ,
                data=filtered_orderlines)

res_model_8 = model_8.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_8.summary())


In [ ]:
KVI_Cluster = dfAllStandard[(dfAllStandard["Cluster"] == 10) | (dfAllStandard["Cluster"] == 17)]["ProductId"].to_list()
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_Cluster)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].max()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_10_17= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVIPrice' ,
                data=filtered_orderlines)

res_model_10_17 = model_10_17.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_10_17.summary())


In [ ]:
invoices_with_kvi = OrderLinesData[OrderLinesData["ProductId"].isin(KVI_list)]["InvoiceId"].unique()
invoice_product_counts = OrderLinesData.groupby('InvoiceId')['ProductId'].nunique()
invoices_with_2_or_more = invoice_product_counts[invoice_product_counts >= 2].index
valid_invoices = set(invoices_with_kvi) & set(invoices_with_2_or_more)
filtered_orderlines = OrderLinesData[OrderLinesData['InvoiceId'].isin(valid_invoices)]
filtered_orderlines["KVI"] = filtered_orderlines["ProductId"].isin(KVI_Cluster).astype(int)
filtered_orderlines = filtered_orderlines.drop(columns=['RegionId', 'InvoiceAttributeA',
       'RegionCurrency', 'CustomerName', 'CustomerDescription',
       'CustomerAttributeA', 'PricePerUnitVat', 'MarginPerUnit', 'TotalCostPerUnit',
       'PurchaseCostPerUnit', 'ShippingCostPerUnit', 'OtherCostPerUnit',
       'FeePercentage', 'VatPercentage', 'OrderLineAttributeA',
       'OrderLineAttributeB', 'OrderLineAttributeC', 'Deals', 'Date Month'])

kvi_prices = filtered_orderlines[filtered_orderlines['ProductId'].isin(KVI_Cluster)].groupby('InvoiceId')['PricePerUnit'].max()

filtered_orderlines = filtered_orderlines.merge(kvi_prices, on='InvoiceId', how='left', suffixes=('', '_KVI'))

filtered_orderlines["KVIPrice"] = filtered_orderlines["PricePerUnit"].where(filtered_orderlines["KVI"] == 1, filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines["KVIPrice"] = filtered_orderlines["KVIPrice"].fillna(filtered_orderlines["PricePerUnit_KVI"])

filtered_orderlines = filtered_orderlines.drop(columns=['PricePerUnit_KVI'])

filtered_orderlines = pd.merge(filtered_orderlines, Products.filter(['ProductId','PriceChangeFrequency','ProductType','Availability']), on='ProductId', how='left')
filtered_orderlines = pd.merge(filtered_orderlines, num_customers, on='ProductId', how='left')

filtered_orderlines = filtered_orderlines.dropna()


filtered_orderlines["PricePerUnit"] = np.log(filtered_orderlines["PricePerUnit"])
filtered_orderlines["KVIPrice"] = np.log(filtered_orderlines["KVIPrice"])
filtered_orderlines["PriceChangeFrequency"] = np.log(filtered_orderlines["PriceChangeFrequency"]+1)
filtered_orderlines["NumCustomers"] = np.log(filtered_orderlines["NumCustomers"])
filtered_orderlines["Quantity"] = np.log(filtered_orderlines["Quantity"])



model_list= smf.ols(formula= 'Quantity ~ PricePerUnit*PriceChangeFrequency +  PricePerUnit*C(ProductType) + PricePerUnit*C(Availability) + PricePerUnit*NumCustomers + PricePerUnit*KVIPrice' ,
                data=filtered_orderlines)

res_model_list = model_list.fit(cov_type='cluster', cov_kwds={'groups': filtered_orderlines['InvoiceId']})
print(res_model_list.summary())
